In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display

load_dotenv(override=True)

True

In [2]:
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_base_url = os.getenv("OPENROUTER_BASE_URL")
deepseek_model_name = os.getenv("DEEPSEEK_MODEL_NAME")

In [23]:
manim_agent_system_prompt = """
You are a professional python manim code writer your task is to get the user's prompt and
generate the manim code for the topic the user wants to learn, your manim script is perfect in
such a way that the animations, visualizations and simulations and teachings will be understood 
even for a layman who watches it.
Note: The manim script when converted to video must take 1 to 2 minutes of duration and ensure the video must be super easy for anyone to be engaged and understand the topic

the output format of your manim code must be like no markdown syntax for wrapping the code or no explanation at all only the code must be the output!

[your manim script here...]
"""

manim_agent = OpenAI(api_key=deepseek_api_key, base_url=deepseek_base_url)

In [25]:
content = "pythagoras theorem"

response = manim_agent.chat.completions.create(
    messages=[
        {'role': 'system', 'content': manim_agent_system_prompt},
        {'role': 'user', 'content': content}
    ],
    model=deepseek_model_name
)

In [28]:
code = response.choices[0].message.content

print(code)

```python
from manim import *

class PythagoreanTheorem(Scene):
    def construct(self):
        # Set colors and styles
        triangle_color = YELLOW
        a_color = GREEN
        b_color = BLUE
        c_color = RED
        square_color = PINK
        fill_opacity = 0.4
        text_scale = 0.7
        
        # Create the right-angled triangle
        triangle = Polygon(
            ORIGIN, 
            3 * RIGHT, 
            3 * RIGHT + 4 * UP, 
            color=triangle_color
        )
        triangle.set_fill(triangle_color, opacity=fill_opacity)
        
        # Label sides of triangle
        a_label = MathTex("a", color=a_color).next_to(triangle.get_bottom(), UP, buff=0.1)
        b_label = MathTex("b", color=b_color).next_to(triangle, LEFT, buff=0.1)
        c_label = MathTex("c", color=c_color).next_to(triangle.get_center(), UP + RIGHT, buff=0.2)
        
        # Create squares on each side
        square_a = Polygon(
            ORIGIN, 
            ORIGIN + 3 *